# Clase 05 - Técnicas para limpiar datos con Pandas

En este cuaderno analizaremos el "menú de técnicas" que usaremos para dejar una base lista para analizar. Vamos a repasar una por una, con ejemplos de casos típicos y cómo encararlos.


## Eliminación de duplicados — drop_duplicates()

Mencionamos dos "trucos" que funcionan casi siempre:

* Si un registro está repetido exactamente igual, `drop_duplicates()` lo elimina y deja uno.
* Si queremos detectar duplicados según algunas columnas (por ejemplo, mismo DNI y mismo email, aunque el resto cambie), usamos `subset=['dni','email']`.

**Recordamos:**

* `keep='first'` o `keep='last'` decide cuál conservar;
* `keep=False` elimina todos los duplicados.

Después de eliminar, conviene contar cuántos quitamos y hacer un pequeño reporte: “Quitamos X duplicados”.

In [1]:
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd

# =========================================================
# Ejemplo de DataFrame con registros duplicados
# =========================================================
data = {
    "dni": [111, 222, 111, 333, 444, 222],   # Hay DNIs repetidos
    "email": [
        "ana@mail.com",
        "juan@mail.com",
        "ana@mail.com",
        "pepe@mail.com",
        "luis@mail.com",
        "JUAN@mail.com"   # Ojo: igual que Juan pero con mayúsculas
    ],
    "edad": [25, 30, 25, 40, 22, 30]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# =========================================================
# 1. Eliminar duplicados EXACTOS
#    (toda la fila es idéntica a otra)
# =========================================================
df_sin_duplicados_exactos = df.drop_duplicates()

print("\n=== Sin duplicados EXACTOS ===")
print(df_sin_duplicados_exactos)

# =========================================================
# 2. Detectar duplicados según un subconjunto de columnas
#    En este caso: DNI y Email
#    Si hay dos filas con mismo DNI y Email, se eliminan
# =========================================================
df_sin_duplicados_dni_email = df.drop_duplicates(subset=["dni", "email"])

print("\n=== Sin duplicados por DNI + Email ===")
print(df_sin_duplicados_dni_email)

# =========================================================
# 3. Control sobre cuál duplicado conservar
#    keep='first' -> deja el primero
#    keep='last'  -> deja el último
#    keep=False   -> elimina TODOS los duplicados
# =========================================================

df_keep_last = df.drop_duplicates(keep="last")
print("\n=== Conservando el ÚLTIMO duplicado ===")
print(df_keep_last)

df_eliminar_todos = df.drop_duplicates(keep=False)
print("\n=== Eliminando TODOS los duplicados ===")
print(df_eliminar_todos)

# =========================================================
# 4. Contar cuántos duplicados había en total
#    Para dejar registro en un "reporte"
# =========================================================
total_filas = len(df)
total_sin_dup = len(df_sin_duplicados_exactos)
duplicados_eliminados = total_filas - total_sin_dup

print(f"\nSe eliminaron {duplicados_eliminados} filas duplicadas.")


=== DataFrame original ===
   dni          email  edad
0  111   ana@mail.com    25
1  222  juan@mail.com    30
2  111   ana@mail.com    25
3  333  pepe@mail.com    40
4  444  luis@mail.com    22
5  222  JUAN@mail.com    30

=== Sin duplicados EXACTOS ===
   dni          email  edad
0  111   ana@mail.com    25
1  222  juan@mail.com    30
3  333  pepe@mail.com    40
4  444  luis@mail.com    22
5  222  JUAN@mail.com    30

=== Sin duplicados por DNI + Email ===
   dni          email  edad
0  111   ana@mail.com    25
1  222  juan@mail.com    30
3  333  pepe@mail.com    40
4  444  luis@mail.com    22
5  222  JUAN@mail.com    30

=== Conservando el ÚLTIMO duplicado ===
   dni          email  edad
1  222  juan@mail.com    30
2  111   ana@mail.com    25
3  333  pepe@mail.com    40
4  444  luis@mail.com    22
5  222  JUAN@mail.com    30

=== Eliminando TODOS los duplicados ===
   dni          email  edad
1  222  juan@mail.com    30
3  333  pepe@mail.com    40
4  444  luis@mail.com    22
5  222 

## Eliminación de caracteres no deseados — str.replace()

Partimos de un DataFrame con nombres "sucios" (tildes, espacios, símbolos, mayúsculas mezcladas) y vamos a poner en práctica algunas formas de limpiarlos. Paso a paso.


**Ejemplos típicos:**
* Quitar espacios extra: `col.str.strip()` (espacios en blanco en los bordes) y `col.str.replace(r'\s+',' ', regex=True)` (espacios internos).
* Homogeneizar separadores de miles o moneda antes de convertir a número (por ej., quitar $ o . de miles).
* Arreglar caracteres específicos (como cambiar `ñ` por `n` si la app de destino no soporta unicode).

Recordemos que `str.replace` acepta expresiones regulares (`regex=True`), útil para limpiar varios símbolos en un solo paso.

In [3]:
import pandas as pd

# =========================================================
# Ejemplo de DataFrame con texto "sucio"
# =========================================================
data = {
    "nombre": [
        " Ana Pérez ",   # Tiene espacios al inicio y al final
        "JUAN!!",        # Símbolos extra
        "luis#",         # Carácter no deseado
        "María-Luisa",   # Guión en medio
        "carl0s"         # Error tipográfico: cero en lugar de 'o'
    ]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# =========================================================
# 1. Eliminar espacios en blanco al inicio y al final
# =========================================================
df["nombre_limpio"] = df["nombre"].str.strip()

print("\n=== Sin espacios al inicio/final ===")
print(df)

# =========================================================
# 2. Eliminar símbolos no deseados (!, #, -)
#    Usamos una expresión regular para quedarnos
#    solo con letras y espacios
# =========================================================
df["nombre_limpio"] = df["nombre_limpio"].str.replace(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]", "", regex=True)

print("\n=== Sin símbolos extra ===")
print(df)

# =========================================================
# 3. Convertir todo a minúsculas para normalizar
# =========================================================
df["nombre_limpio"] = df["nombre_limpio"].str.lower()

print("\n=== En minúsculas ===")
print(df)

# =========================================================
# 4. Corrección manual de errores comunes
#    (ejemplo: reemplazar '0' por 'o')
# =========================================================
df["nombre_limpio"] = df["nombre_limpio"].str.replace("0", "o")

print("\n=== Corrección de errores tipográficos ===")
print(df)

# =========================================================
# Resultado: columna 'nombre_limpio' lista para usar
# =========================================================
print("\n=== Resultado final ===")
print(df[["nombre", "nombre_limpio"]])


=== DataFrame original ===
        nombre
0   Ana Pérez 
1       JUAN!!
2        luis#
3  María-Luisa
4       carl0s

=== Sin espacios al inicio/final ===
        nombre nombre_limpio
0   Ana Pérez      Ana Pérez
1       JUAN!!        JUAN!!
2        luis#         luis#
3  María-Luisa   María-Luisa
4       carl0s        carl0s

=== Sin símbolos extra ===
        nombre nombre_limpio
0   Ana Pérez      Ana Pérez
1       JUAN!!        JUAN!!
2        luis#         luis#
3  María-Luisa    MaríaLuisa
4       carl0s         carls

=== En minúsculas ===
        nombre nombre_limpio
0   Ana Pérez      ana pérez
1       JUAN!!        juan!!
2        luis#         luis#
3  María-Luisa    maríaluisa
4       carl0s         carls

=== Corrección de errores tipográficos ===
        nombre nombre_limpio
0   Ana Pérez      ana pérez
1       JUAN!!        juan!!
2        luis#         luis#
3  María-Luisa    maríaluisa
4       carl0s         carls

=== Resultado final ===
        nombre nombre_limpio


## Corrección de tipos de datos — astype()

Vimos que si intentamos realizar un análisis numérico con columnas de tipo "texto", falla o da resultados "raros".


* Si una columna debería ser numérica (precio, edad), la convertimos al tipo numérico.
* Cuando hay valores "sucios", `astype(float)` puede fallar. En esos casos, `pd.to_numeric(col, errors='coerce')` transforma lo inválido en `NaN` y nos permite continuar.
* Para fechas, `pd.to_datetime(col, dayfirst=True/False)` unifica formatos y (casi siempre) resuelve ambigüedades (útil con dd/mm vs mm/dd).
* Documentamos las decisiones: "forzamos a numérico y quedaron N valores como NaN por caracteres no válidos".


Veamos algunos ejemplos simples, pero claros que ilustran esta tercera técnica, la **corrección de tipos de datos**:

En el código vemos tres casos distintos:

* Números como texto,
* Valores inválidos, y
* Fechas en formatos diferentes.

---

**Nota**:
`errors='coerce'` le dice a Pandas: "Si encontrás un valor que no podés convertir al tipo solicitado, en lugar de tirar un error, transformalo en `NaN` (valor nulo)."

In [2]:

import pandas as pd

# =========================================================
# Ejemplo de DataFrame con tipos de datos incorrectos
# Mezcla de números y texto en "edad"
# Coma como decimal y texto "NaN" en "precio"
# Distintos formatos en "fecha"
# =========================================================
data = {
    "edad": ["25", "30", "treinta", "40"],
    "precio": ["100.5", "200,75", "300", "NaN"],
    "fecha": ["01/02/2023", "2023-03-15", "15/04/2023", "03-05-2023"]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df.dtypes)  # Tipos iniciales (todos 'object')
print(df)

# =========================================================
# 1. Conversión de 'edad' a numérico
#    - Usamos to_numeric con errors='coerce'
#    - Los valores no convertibles pasan a NaN
# =========================================================
df["edad"] = pd.to_numeric(df["edad"], errors="coerce")

print("\n=== Columna 'edad' convertida a numérico ===")
print(df)
print(df.dtypes)

# =========================================================
# 2. Conversión de 'precio'
#    - Primero reemplazamos coma por punto
#    - Luego convertimos a float
# =========================================================
# Homogeneizamos
df["precio"] = df["precio"].str.replace(",", ".", regex=False)
df["precio"] = pd.to_numeric(df["precio"], errors="coerce")

print("\n=== Columna 'precio' convertida a float ===")
print(df)
print(df.dtypes)

# =========================================================
# 3. Conversión de 'fecha' a datetime
#    - Usamos dayfirst=True porque hay casos con dd/mm/yyyy
# =========================================================
df["fecha"] = pd.to_datetime(df["fecha"], dayfirst=True, errors="coerce")

print("\n=== Columna 'fecha' convertida a datetime ===")
print(df)
print(df.dtypes)

# =========================================================
# 4. Resumen de lo hecho
# =========================================================
print("\nResumen:")
print("- 'edad' ahora es numérica, con NaN donde había texto inválido.")
print("- 'precio' se normalizó a float, corrigiendo comas y NaN textuales.")
print("- 'fecha' quedó en formato datetime homogéneo.")


=== DataFrame original ===
edad      object
precio    object
fecha     object
dtype: object
      edad  precio       fecha
0       25   100.5  01/02/2023
1       30  200,75  2023-03-15
2  treinta     300  15/04/2023
3       40     NaN  03-05-2023

=== Columna 'edad' convertida a numérico ===
   edad  precio       fecha
0  25.0   100.5  01/02/2023
1  30.0  200,75  2023-03-15
2   NaN     300  15/04/2023
3  40.0     NaN  03-05-2023
edad      float64
precio     object
fecha      object
dtype: object

=== Columna 'precio' convertida a float ===
   edad  precio       fecha
0  25.0  100.50  01/02/2023
1  30.0  200.75  2023-03-15
2   NaN  300.00  15/04/2023
3  40.0     NaN  03-05-2023
edad      float64
precio    float64
fecha      object
dtype: object

=== Columna 'fecha' convertida a datetime ===
   edad  precio      fecha
0  25.0  100.50 2023-02-01
1  30.0  200.75        NaT
2   NaN  300.00 2023-04-15
3  40.0     NaN        NaT
edad             float64
precio           float64
fecha     date

¿Qué muestra el ejemplo anterior?:

* **Edad**: el valor "treinta" no se pudo convertir, y quedó como NaN.
* **Precio**: los valores con coma decimal (200,75) fueron normalizados a 200.75.
* **Fecha**: todas se transforman al mismo tipo datetime64, resolviendo diferencias de formato.



## Manejo de datos faltantes — fillna() / dropna()

Mencionamos dos estrategias básicas:

* Eliminar filas incompletas cuando el campo es crítico y el porcentaje de faltantes es bajo: `df.dropna(subset=['edad'])`.

* Imputar cuando eliminar distorsionaría la muestra:
	* Numéricos: media o mediana `(df['edad'].fillna(df['edad'].median()))`.
	* Categóricos: moda o una etiqueta “desconocido”.

Sugerencia docente: antes de decidir, midan cuánto falta: `df.isna().mean()` para ver el porcentaje de nulos por columna y justificar la elección.

---

**En el ejemplo:**
* Primero vemos los porcentajes de nulos para decidir.
* Luego eliminamos filas con edad nula cuando es un campo crítico.
* Mostramos cómo imputar en variables numéricas (mediana) y categóricas (moda o “desconocido”).
* El dataset resultante queda completo y consistente, listo para análisis sin perder información valiosa.


In [4]:
import pandas as pd
import numpy as np

# =========================================================
# Ejemplo de DataFrame con valores faltantes
# =========================================================
data = {
    "nombre": ["Ana", "Juan", "Luis", "María", None],
    "edad": [25, None, 30, None, 40],
    "ciudad": ["Buenos Aires", "Córdoba", None, "Rosario", "Rosario"]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# =========================================================
# 1. Medir el porcentaje de nulos por columna
#    (Nos puede ayudar a decidir qué estrategia usar)
# =========================================================
print("\n=== Porcentaje de nulos por columna ===")
print(df.isna().mean() * 100)

# =========================================================
# 2. Eliminar filas incompletas en una columna crítica
#    (ejemplo: 'edad' es fundamental para el análisis)
# =========================================================
# Ojo: df_drop = df.dropna() sin subset elimina cualquier
# fila que tenga al menos un NaN, en cualquier columna.
df_drop = df.dropna(subset=["edad"])
print("\n=== Eliminando filas con edad nula ===")
print(df_drop)

# =========================================================
# 3. Imputación en columna numérica (edad)
#    - Usamos la mediana para evitar distorsión por outliers
# =========================================================
df_fill = df.copy()
df_fill["edad"] = df_fill["edad"].fillna(df_fill["edad"].median())

print("\n=== Imputando edad con la mediana ===")
print(df_fill)

# =========================================================
# 4. Imputación en columna categórica (ciudad)
#    - Usamos la moda (valor más frecuente)
#    - También podría usarse una etiqueta ('desconocido')
# =========================================================
moda_ciudad = df_fill["ciudad"].mode()[0]  # valor más frecuente
df_fill["ciudad"] = df_fill["ciudad"].fillna(moda_ciudad)

print("\n=== Imputando ciudad con la moda ===")
print(df_fill)

# =========================================================
# 5. Opcional: imputar valores faltantes en 'nombre'
#    - Etiqueta genérica para no perder filas
# =========================================================
df_fill["nombre"] = df_fill["nombre"].fillna("desconocido")

print("\n=== Imputando nombre con etiqueta 'desconocido' ===")
print(df_fill)


=== DataFrame original ===
  nombre  edad        ciudad
0    Ana  25.0  Buenos Aires
1   Juan   NaN       Córdoba
2   Luis  30.0          None
3  María   NaN       Rosario
4   None  40.0       Rosario

=== Porcentaje de nulos por columna ===
nombre    20.0
edad      40.0
ciudad    20.0
dtype: float64

=== Eliminando filas con edad nula ===
  nombre  edad        ciudad
0    Ana  25.0  Buenos Aires
2   Luis  30.0          None
4   None  40.0       Rosario

=== Imputando edad con la mediana ===
  nombre  edad        ciudad
0    Ana  25.0  Buenos Aires
1   Juan  30.0       Córdoba
2   Luis  30.0          None
3  María  30.0       Rosario
4   None  40.0       Rosario

=== Imputando ciudad con la moda ===
  nombre  edad        ciudad
0    Ana  25.0  Buenos Aires
1   Juan  30.0       Córdoba
2   Luis  30.0       Rosario
3  María  30.0       Rosario
4   None  40.0       Rosario

=== Imputando nombre con etiqueta 'desconocido' ===
        nombre  edad        ciudad
0          Ana  25.0  Buenos 

## Normalización de datos

Buscamos consistencia para agrupar y comparar sin “ruido”:

* **Texto**: `str.lower()` para minúsculas, `str.strip()` para eliminar espacios en los extremos de las cadenas, y reemplazos de variantes (“Analista”, “analista”, “Analist”) a una lista controlada (diccionario de mapeo).
* **Códigos y categorías**: definir dominios válidos (por ejemplo, provincias ISO o categorías de producto oficiales) y mapear todo a ese estándar.
* **Fechas y números:** elegir un formato único y sostenerlo en todo el dataset.

Esta normalización evita que un mismo concepto quede partido en varias etiquetas.


Veamos un ejemplo donde aparezcan inconsistencias en texto, categorías y fechas, y que podemos hacer para normalizarlas.

In [ ]:
import pandas as pd

# =========================================================
# Ejemplo de DataFrame con datos "desnormalizados" :P
# =========================================================
data = {
    "cargo": ["Analista", "analista ", "Analist", "ANALISTA", "Gerente"],
    "provincia": ["Bs As", "CABA", "Buenos Aires", "Santa Fe", "Santa Fé"],
    "fecha": ["01/02/2023", "2023/03/15", "09-abril-2023", "01/12/2024", "2023/05/10"]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# =========================================================
# 1. Normalización de texto en columna 'cargo'
#    - Pasamos todo a minúsculas
#    - Eliminamos espacios extras
#    - Reemplazamos variantes a un valor estándar con map
# =========================================================
df["cargo_norm"] = (
    df["cargo"]
    .str.lower() # Minus.
    .str.strip() # Quitamos los espacios
    .replace({"analist": "analista"})  # mapeo de variantes
)

print("\n=== Columna 'cargo' normalizada ===")
print(df[["cargo", "cargo_norm"]])

# =========================================================
# 2. Normalización de categorías en 'provincia'
#    - Definimos un diccionario de equivalencias a un estándar
# =========================================================
map_provincia = {
    "bs as": "Buenos Aires",
    "buenos aires": "Buenos Aires",
    "caba": "CABA",
    "santa fé": "Santa Fe"
}

df["provincia_norm"] = (
    df["provincia"]
    .str.lower()
    .str.strip()
    .replace(map_provincia)
    .str.title() # Primera en mayusculas (Aasdas asdas ) (Aasd Asdasd)
)

print("\n=== Columna 'provincia' normalizada ===")
print(df[["provincia", "provincia_norm"]])

# =========================================================
# 3. Normalización de fechas
#    - Usamos pd.to_datetime con dayfirst=True
#    - Unificamos distintos formatos a un único datetime
# =========================================================
df["fecha_norm"] = pd.to_datetime(df["fecha"], dayfirst=True, errors="coerce")

print("\n=== Columna 'fecha' normalizada ===")
print(df[["fecha", "fecha_norm"]])

# =========================================================
# Resultado final
# =========================================================
print("\n=== DataFrame completo con normalización ===")
print(df)


=== DataFrame original ===
       cargo     provincia          fecha
0   Analista         Bs As     01/02/2023
1  analista           CABA     2023/03/15
2    Analist  Buenos Aires  09-abril-2023
3   ANALISTA      Santa Fe     01/12/2024
4    Gerente      Santa Fé     2023/05/10

=== Columna 'cargo' normalizada ===
       cargo cargo_norm
0   Analista   analista
1  analista    analista
2    Analist   analista
3   ANALISTA   analista
4    Gerente    gerente

=== Columna 'provincia' normalizada ===
      provincia provincia_norm
0         Bs As   Buenos Aires
1          CABA           Caba
2  Buenos Aires   Buenos Aires
3      Santa Fe       Santa Fe
4      Santa Fé       Santa Fe

=== Columna 'fecha' normalizada ===
           fecha fecha_norm
0     01/02/2023 2023-02-01
1     2023/03/15        NaT
2  09-abril-2023        NaT
3     01/12/2024 2024-12-01
4     2023/05/10        NaT

=== DataFrame completo con normalización ===
       cargo     provincia          fecha cargo_norm provincia

¿Qué logramos con los pasos realizados en el ejemplo?

* En *cargo*, todas las variantes terminan convertidas en "analista", de forma homogénea.
* En **provincia**, distintas formas de escribir lo mismo se transforman a un diccionario estándar.
* En **fecha**, todos los valores quedan en un único formato datetime64, listo para ser utilizado cálculos y comparaciones.

En este caso, el resultado es (siendo conservadores) mediocre: **casi todas las fechas quedaron como NaT.** Esto ocurre por que la columna *fecha* mezcla muchos formatos raros (con nombres de meses en español, barras, guiones, abreviados, etc.), y `pd.to_datetime` es excelente, pero...no hace milagros.

Si tenemos la mala suerte de encontrarnos con un dataset tan heterogeneo, podemos:

* Unificar formatos antes de convertir: reemplazar nombres de meses en español por números.
* Usar dayfirst=True si los datos están en formato latino (dd/mm/yyyy).
* Probar con infer_datetime_format=True (en versiones anteriores de pandas funcionaba mejor).
* Para casos muy particulares o puntuales, se puede aplicar una función personalizada con `str.replace`.
* Y si todo falla....escribir un script Python con un bucle, "buscar" en cada fecha los separadores, y "armar" la cadena con la fecha de manera artesanal. Luego si, aplicar `pd.to_datetime`.

## Filtrado de datos

Es el sexto paso que vimos en la presentación de diapositivas. Algunas ideas para filtrar los datos:



* Con máscaras booleanas: `df[df['edad'] > 25]`, o combinaciones con `&` y `|`.
* Con `query()`: más legible para condiciones complejas, por ejemplo `df.query("precio > 0 and categoria == 'ropa'")`.
* Filtrar también puede ser parte del proceso de "sanar" un dataframe: descartar outliers imposibles (edad negativa) o registros de prueba. La idea es quedarnos con un subconjunto coherente con el objetivo del análisis.


In [ ]:
import pandas as pd

# =========================================================
# Ejemplo de DataFrame con datos para filtrar
# Hay menores, edad negativa y un outlier extremo
# Hay precio=0 y precios negativos
# =========================================================
data = {
    "nombre": ["Ana", "Juan", "Luis", "María", "Pedro", "TestUser"],
    "edad": [25, 17, -5, 40, 70, 999],
    "categoria": ["ropa", "tecnología", "ropa", "alimentos", "ropa", "test"],
    "precio": [1500, 0, 3000, 500, -100, 50]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# =========================================================
# 1. Filtro con máscara booleana
#    (Dejamos solo a personas mayores de 25 años)
# =========================================================
df_mayores = df[df["edad"] > 25]
print("\n=== Personas con edad > 25 ===")
print(df_mayores)

# =========================================================
# 2. Filtro con múltiples condiciones
#    (Dejamos Edad mayor a 18 Y categoría igual a 'ropa')
# =========================================================
df_condiciones = df[(df["edad"] > 18) & (df["categoria"] == "ropa")]
print("\n=== Mayores de 18 en categoría 'ropa' ===")
print(df_condiciones)

# =========================================================
# 3. Filtro con query()
#    (Dejamos los productos de la categoría ropa con precio > 0)
# =========================================================
df_query = df.query("precio > 0 and categoria == 'ropa'")
print("\n=== query(): ropa con precio > 0 ===")
print(df_query)

# =========================================================
# 4. Filtrado completo
#    - Quitamos edades negativas y mayores a 120
#    - Quitamos precios negativos
#    - Quitamos registros de prueba
# =========================================================
df_sano = df[
    (df["edad"].between(0, 120)) &
    (df["precio"] > 0) &
    (df["categoria"] != "test")
]

print("\n=== Filtrado Final ===")
print(df_sano)



=== DataFrame original ===
     nombre  edad   categoria  precio
0       Ana    25        ropa    1500
1      Juan    17  tecnología       0
2      Luis    -5        ropa    3000
3     María    40   alimentos     500
4     Pedro    70        ropa    -100
5  TestUser   999        test      50

=== Personas con edad > 25 ===
     nombre  edad  categoria  precio
3     María    40  alimentos     500
4     Pedro    70       ropa    -100
5  TestUser   999       test      50

=== Mayores de 18 en categoría 'ropa' ===
  nombre  edad categoria  precio
0    Ana    25      ropa    1500
4  Pedro    70      ropa    -100

=== query(): ropa con precio > 0 ===
  nombre  edad categoria  precio
0    Ana    25      ropa    1500
2   Luis    -5      ropa    3000

=== Filtrado Final ===
  nombre  edad  categoria  precio
0    Ana    25       ropa    1500
3  María    40  alimentos     500


Con estas seis técnicas de trabajo —duplicados, caracteres, tipos, faltantes, normalización y filtrado— logramos pasar de datos desprolijos a una base confiable.

A partir de este trabajo preliminar, a veces duro y complejo, nos aseguramos que cualquier métrica o visualización que obtangomos sea mucho más robusta.
